In [ ]:
import os
import sys
from wayne_utils import load_data, save_data
_ROOT_PATH = "workspace/projects/TKGT"
sys.path.insert( 0, _ROOT_PATH )
_Data_path = os.path.join( _ROOT_PATH, "data/CPL/dynamic")
version_dir = os.path.join( _ROOT_PATH, "test/CPL_dynamic/v1")

prefix_list = []
for entity in [ "法院", "原告", "被告"]:
    for field in [ "姓名名称", "借款凭证", "约定的借款金额", "约定的还款日期或借款期限", "约定的利息", "约定的逾期利息", "约定的违约金"]:
        if entity == "被告" and field != "姓名名称":
            continue
        if field == "姓名名称":
            prefix_list.append( entity )
        else:
            pre = "法院裁定_" if entity == "法院" else "原告诉称_"
            prefix_list.append( pre+ field)
prefix_list

['法院',
 '法院裁定_借款凭证',
 '法院裁定_约定的借款金额',
 '法院裁定_约定的还款日期或借款期限',
 '法院裁定_约定的利息',
 '法院裁定_约定的逾期利息',
 '法院裁定_约定的违约金',
 '原告',
 '原告诉称_借款凭证',
 '原告诉称_约定的借款金额',
 '原告诉称_约定的还款日期或借款期限',
 '原告诉称_约定的利息',
 '原告诉称_约定的逾期利息',
 '原告诉称_约定的违约金',
 '被告']

In [7]:
def get_number( line, prompt):
    if "：[]" in prompt:
        return 0
    for i in range( 6):
        if str(i) in line:
            return i
    return 0

In [14]:
model_list = [
    "ChatGLM3-6B",
    "Qwen1.5-7B-Chat",
    "Chinese-Mistral-7B-Instruct-v0.1",
    "Baichuan2-7B-Chat",
    "Qwen2.5-0.5B",
    "CPL_dynamic_counter_ChatGLM3-6B-3epoch",
    "CPL_dynamic_counter_Qwen1.5-7B-Chat-3epoch",
    "CPL_dynamic_counter_Chinese-Mistral-7B-Instruct-v0.1-3epoch",
    "CPL_dynamic_counter_Chinese-Mistral-7B-Instruct-v0.1-3epoch_no_rag",
    "CPL_dynamic_counter_Chinese-Mistral-7B-Instruct-v0.1-3epoch_hybrid_rag"
]
scopes = model_list
scopes

['ChatGLM3-6B',
 'Qwen1.5-7B-Chat',
 'Chinese-Mistral-7B-Instruct-v0.1',
 'Baichuan2-7B-Chat',
 'Qwen2.5-0.5B',
 'CPL_dynamic_counter_ChatGLM3-6B-3epoch',
 'CPL_dynamic_counter_Qwen1.5-7B-Chat-3epoch',
 'CPL_dynamic_counter_Chinese-Mistral-7B-Instruct-v0.1-3epoch',
 'CPL_dynamic_counter_Chinese-Mistral-7B-Instruct-v0.1-3epoch_no_rag',
 'CPL_dynamic_counter_Chinese-Mistral-7B-Instruct-v0.1-3epoch_hybrid_rag']

In [ ]:
from evaluation.CountCPL.evaluate import Eval
version_dir = "workspace/projects/TKGT/test/CPL_static/v2_dynamic_baseline"
for model_name in scopes:
    predict_list = load_data( os.path.join( version_dir, f"counter_predict_list_{model_name}.pickle"), "pickle")
    prompt_list = load_data( os.path.join( version_dir, "counter_prompt_list.pickle"), "pickle")    # 111
    label_count_list = load_data( os.path.join( _ROOT_PATH, "evaluation/CountCPL/label_count_list_test.json"), "json")
    predict_count_list = []
    for i in range(len(predict_list)):
        temp_dict = {}
        predicts = predict_list[i]
        prompts = prompt_list[i]
        if len(predicts) != len(prefix_list):
            raise Exception( f"预测结果长度不对{len(predicts)}")
        for j in range( len(predicts)):
            temp_dict[prefix_list[j]] = get_number( predicts[j], prompts[j] )
        predict_count_list.append( temp_dict )
    save_data( predict_count_list, os.path.join( version_dir, f"counter_predict_list_final_{model_name}.pickle"))
    
    aadf, error_rate = Eval( predict_count_list, label_count_list )
    print( model_name )
    print( aadf.describe().loc["mean"])

test 211 tables for sss
Error rate = 26.540284360189574 34.20221169036335 57.74091627172196 42.08530805687204
ChatGLM3-6B
Easy-RMSE       0.443976
Easy-MAE        0.210900
Easy-EM        63.270142
Medium-RMSE     0.419424
Medium-MAE      0.209321
Medium-EM      67.101106
Hard-RMSE       0.642716
Hard-MAE        0.342812
Hard-EM        78.870458
AVG-RMSE        0.587745
AVG-MAE         0.263033
AVG-EM         71.042654
Name: mean, dtype: float64
test 211 tables for sss
Error rate = 4.739336492890995 21.248025276461295 31.1216429699842 21.895734597156398
Qwen1.5-7B-Chat
Easy-RMSE       0.055778
Easy-MAE        0.023697
Easy-EM        52.369668
Medium-RMSE     0.269579
Medium-MAE      0.115324
Medium-EM      60.624013
Hard-RMSE       0.389793
Hard-MAE        0.180885
Hard-EM        65.560821
AVG-RMSE        0.354701
AVG-MAE         0.123223
AVG-EM         60.947867
Name: mean, dtype: float64
test 211 tables for sss
Error rate = 2.0537124802527646 48.1042654028436 46.52448657187994 38.2622